In [1]:
!pip install -q opencv-python
!pip install -q deepface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.6/58.6 kB 937.8 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 8.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.2/80.2 kB 7.2 MB/s eta 0:00:00


In [3]:
from deepface import DeepFace
from google.colab.patches import cv2_imshow
import cv2
from IPython.display import display, Javascript, clear_output
from google.colab.output import eval_js
import base64
import numpy as np
from PIL import Image
import io
import datetime
import time

# Create round variables
round_num = 1
start_time = datetime.datetime.now()
end_time = None

leaderboard_file = 'leaderboard.txt'
leaderboard = []

# Check if leaderboard exists
try:
    with open(leaderboard_file, 'r') as file:
        leaderboard = [float(time) for time in file.readlines()]
except FileNotFoundError:
    pass

# Change button text based on round number
def get_button_text(round_num):
    if round_num == 1:
        return "Round 1: Capture picture of a happy face"
    elif round_num == 2:
        return "Round 2: Capture picture of an angry face"
    elif round_num == 3:
        return "Round 3: Capture picture of a surprised face"
    elif round_num == 4:
        return "You won!"
    else:
        return "Something went wrong"

# Capture webcam in Google Colab
def colab_camera_capture():
    js_code = f'''
    async function takePhoto(quality) {{
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = '{get_button_text(round_num)}';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({{'video': true}});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }}
    '''

    display(Javascript(js_code))
    data = eval_js('takePhoto({})')
    return data

# Exit game when above round 3
while round_num <= 3:
    data = colab_camera_capture()
    image = Image.open(io.BytesIO(base64.b64decode(data.split(',')[1])))
    frame = np.array(image)
    frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

    result = DeepFace.analyze(frame, actions=['emotion'])

    # Find dominant emotion
    emotion = result[0]['dominant_emotion']

    # Show emotion
    cv2.putText(frame, f'Emotion detected: {emotion}', (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
    # Show round number
    cv2.putText(frame, f'Round: {round_num} of 3', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

    # Change text based on round number
    if round_num == 1:
        if emotion == 'happy':
            cv2.putText(frame, 'Completed Round 1', (10, 110), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
            round_num += 1
        else:
            cv2.putText(frame, 'Failed to detect a happy face!', (10, 110), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)

    elif round_num == 2:
        if emotion == 'angry':
            cv2.putText(frame, 'Completed Round 2', (10, 110), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
            round_num += 1
        else:
            cv2.putText(frame, 'Failed to detect an angry face!', (10, 110), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)

    elif round_num == 3:
        if emotion == 'surprise':
            end_time = datetime.datetime.now()
            elapsed_time = end_time - start_time
            cv2.putText(frame, f'You won! Game Over', (10, 110), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
            cv2.putText(frame, f'Time taken: {elapsed_time}', (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
            leaderboard.append(elapsed_time.total_seconds())
            round_num += 1
        else:
            cv2.putText(frame, 'Failed to detect a surprised face!', (10, 110), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)

    cv2_imshow(frame)

    # Wait 3 seconds before clearing
    time.sleep(3)
    clear_output(wait=True)

# Print leaderboard
print("Leaderboard:")
for i, time in enumerate(sorted(leaderboard)):
    print(f"{i + 1}. {time} seconds")

# Save leaderboard
with open(leaderboard_file, 'w') as file:
    for time in leaderboard:
        file.write(f"{time}\n")

cv2.destroyAllWindows()

Leaderboard:
1. 30.760017 seconds
2. 110.144994 seconds
